# NVDI PDP local Estimation

In [1]:
%pwd

'/home/pj24001881/share/DP15_Article/12_PyCode'

In [2]:
%cd ..

/home/pj24001881/share/DP15_Article


/home/pj24001881/ku40001335/miniconda3/envs/tf/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import Package 

In [10]:
from glob import glob
from joblib import dump, load
import joblib
from multiprocessing import Pool, cpu_count
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from tqdm import tqdm

In [4]:
def process_row(line_row):
    left = ori_df.iloc[line_row, 29]
    right = ori_df.iloc[line_row, 30]
    down = ori_df.iloc[line_row, 31]
    up = ori_df.iloc[line_row, 32]

    this_point_df = pdp_df[(pdp_df['lat'] > down) & (pdp_df['lat'] < up) &
                           (pdp_df['lon'] > left) & (pdp_df['lon'] < right)]
    this_point_df = this_point_df[['incre', 'Pred_y_change']]

    if this_point_df.empty:
        return None

    pdp_local = this_point_df.groupby('incre')['Pred_y_change'].mean().reset_index()

    if len(pdp_local) < 2:
        return None

    X = pdp_local[['incre']]
    y = pdp_local['Pred_y_change']

    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)

    # Standard Error of Coefficient
    n = len(y)
    residuals = y - y_pred
    residual_sum_of_squares = np.sum(residuals ** 2)
    x_var = np.sum((X['incre'] - X['incre'].mean()) ** 2)
    se_beta = np.sqrt(residual_sum_of_squares / (n - 2)) / np.sqrt(x_var) if n > 2 and x_var > 0 else np.nan

    lat = ori_df.iloc[line_row, 1]
    lon = ori_df.iloc[line_row, 2]

    return [lat, lon, model.coef_[0], model.intercept_, r2, se_beta]


## Load Data

### Connection NDVI

In [5]:
ori_df_list = []
pdp_df_list = []

for i in list(range(1, 11)):
    X_pdp = pd.read_parquet(f'03_Results/PdpLocal/PDP_Local_NDVI_{i}.parquet')
    length = int(X_pdp.shape[0]/41)
    print(length)
    ori_df_list.append(X_pdp.iloc[:length, :])
    
    y_ori_pred = X_pdp.iloc[:length, -1].to_list() * 41

    value_list = []
    for i in list(range(0, 41)):
        value_list = value_list + [0.05 * i] * length
    
    X_pdp['incre'] = value_list

    X_pdp['Pred_y_change'] = np.array(X_pdp['Pred_y']) - np.array(y_ori_pred)

    pdp_df_list.append(X_pdp)

38318
38318
38318
38317
38317
38317
38317
38317
38317
38317


In [6]:
pdp_df = pd.concat(pdp_df_list)
ori_df = pd.concat(ori_df_list)

In [7]:
pdp_df.shape

(15710093, 36)

In [8]:
ori_df.shape

(383173, 34)

### Build Local Connection

In [11]:
with Pool(processes=cpu_count()-10) as pool:
     line_result_df = list(tqdm(pool.imap(process_row, range(ori_df.shape[0])), total=ori_df.shape[0]))
line_result_df = [res for res in line_result_df if res is not None]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 383173/383173 [13:00<00:00, 491.24it/s]


In [12]:
result_df = pd.DataFrame(line_result_df, columns = ['lat', 'lon', 'coeff', 'intercept', 'r2', 'SE_coeff'])

In [14]:
result_df.to_excel('13_Results/PdpConnectionCoef/PdpConnection_NDVI_v2.xlsx')

### Connection NTL

In [15]:
ori_df_list = []
pdp_df_list = []

for i in list(range(1, 11)):
    X_pdp = pd.read_parquet(f'03_Results/PdpLocal/PDP_Local_NTL_{i}.parquet')
    length = int(X_pdp.shape[0]/41)
    print(length)
    ori_df_list.append(X_pdp.iloc[:length, :])
    
    y_ori_pred = X_pdp.iloc[:length, -1].to_list() * 41

    value_list = []
    for i in list(range(0, 41)):
        value_list = value_list + [0.05 * i] * length
    
    X_pdp['incre'] = value_list

    X_pdp['Pred_y_change'] = np.array(X_pdp['Pred_y']) - np.array(y_ori_pred)

    pdp_df_list.append(X_pdp)

38318
38318
38318
38317
38317
38317
38317
38317
38317
38317


In [16]:
pdp_df = pd.concat(pdp_df_list)
ori_df = pd.concat(ori_df_list)

In [17]:
pdp_df.shape

(15710093, 36)

In [18]:
ori_df.shape

(383173, 34)

### Build Local Connection

In [19]:
with Pool(processes=cpu_count()-10) as pool:
     line_result_df = list(tqdm(pool.imap(process_row, range(ori_df.shape[0])), total=ori_df.shape[0]))
line_result_df = [res for res in line_result_df if res is not None]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 383173/383173 [13:50<00:00, 461.45it/s]


In [20]:
result_df = pd.DataFrame(line_result_df, columns = ['lat', 'lon', 'coeff', 'intercept', 'r2', 'SE_coeff'])

In [27]:
result_df.to_excel('13_Results/PdpConnectionCoef/PdpConnection_NTL_v2.xlsx')

### Connection income

In [28]:
ori_df_list = []
pdp_df_list = []

for i in list(range(1, 11)):
    X_pdp = pd.read_parquet(f'03_Results/PdpLocal/PDP_Local_income_indiv_{i}.parquet')
    length = int(X_pdp.shape[0]/41)
    print(length)
    ori_df_list.append(X_pdp.iloc[:length, :])
    
    y_ori_pred = X_pdp.iloc[:length, -1].to_list() * 41

    value_list = []
    for i in list(range(0, 41)):
        value_list = value_list + [0.05 * i] * length
    
    X_pdp['incre'] = value_list

    X_pdp['Pred_y_change'] = np.array(X_pdp['Pred_y']) - np.array(y_ori_pred)

    pdp_df_list.append(X_pdp)

38318
38318
38318
38317
38317
38317
38317
38317
38317
38317


In [29]:
pdp_df = pd.concat(pdp_df_list)
ori_df = pd.concat(ori_df_list)

In [30]:
pdp_df.shape

(15710093, 36)

In [31]:
ori_df.shape

(383173, 34)

### Build Local Connection

In [33]:
with Pool(processes=cpu_count()-10) as pool:
     line_result_df = list(tqdm(pool.imap(process_row, range(ori_df.shape[0])), total=ori_df.shape[0]))
line_result_df = [res for res in line_result_df if res is not None]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 383173/383173 [15:49<00:00, 403.52it/s]


In [34]:
result_df = pd.DataFrame(line_result_df, columns = ['lat', 'lon', 'coeff', 'intercept', 'r2', 'SE_coeff'])

In [35]:
result_df.to_excel('13_Results/PdpConnectionCoef/PdpConnection_income_indiv_v2.xlsx')